In [17]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, exceptions
import dash_bootstrap_components as dbc  # Import Dash Bootstrap Components
import numpy as np
from PIL import Image

# Load your data
df_radar = pd.read_csv("CleanedData/player_radar.csv")
df_stats = pd.read_csv("CleanedData/player_stats_cleaned.csv")

# Merge dataframes on 'player' after normalizing the 'player' column
df_merged = pd.merge(df_radar, df_stats, on='player', how='outer')

# Calculate performance metrics
df_merged['passing_commulative_performance'] = (
    0.05 * df_merged['passes_completed'] +
    1.5 * df_merged['assists_x'] +
    1.5 * df_merged['assisted_shots'] +
    0.2 * df_merged['passes_into_final_third'] +
    0.4 * df_merged['passes_into_penalty_area'] +
    0.3 * df_merged['crosses_into_penalty_area'] +
    0.1 * df_merged['progressive_passes']
)

df_merged['shooting_commulative_performance'] = (
    3 * df_merged['goals_x'] +
    0.3 * df_merged['shots_on_target'] +
    0.1 * df_merged['shots'] +
    0.2 * df_merged['shots_free_kicks'] +
    df_merged['pens_made_x']
)

df_merged['defence_commulative_performance'] = (
    -0.2 * df_merged['dribbled_past'] -
    df_merged['errors'] +
    0.5 * df_merged['blocks'] +
    0.75 * df_merged['blocked_shots'] +
    0.75 * df_merged['blocked_passes'] +
    df_merged['tackles_interceptions'] +
    0.2 * df_merged['clearances']
)

df_merged['possession_commulative_performance'] = (
    0.05 * df_merged['touches'] +
    0.2 * df_merged['touches_att_3rd'] +
    0.5 * df_merged['touches_att_pen_area'] +
    1 * df_merged['dribbles_completed'] +
    0.1 * df_merged['progressive_passes_received'] +
    -0.2 * df_merged['miscontrols'] +
    -0.5 * df_merged['dispossessed']
)

df_merged['total_performance'] = (
    df_merged['passing_commulative_performance'] +
    df_merged['shooting_commulative_performance'] +
    df_merged['defence_commulative_performance'] +
    df_merged['possession_commulative_performance']
)

# Determine the best player based on total performance
best_player_overall = df_merged.loc[df_merged['total_performance'].idxmax()]
default_team = 'Argentina'  # Set to 'Argentina' to show Argentina players initially
default_highlighted_player = best_player_overall['player']

# Calculate the average performance metrics for all players
average_performance = df_merged[['defence_commulative_performance', 'passing_commulative_performance', 'shooting_commulative_performance', 'possession_commulative_performance']].mean()

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    style={'backgroundColor': '#9bc2cf', 'minHeight': '100vh'},  # Set light blue background for the entire page
    children=[
        dbc.Row([
            dbc.Col(html.Div([
                html.Label('Select team:', style={'fontWeight': 'bold', 'color': 'black'}),
                dcc.Dropdown(
                    id='team-dropdown',
                    options=[{'label': 'No Team', 'value': 'No Team'}] + [{'label': team, 'value': team} for team in df_merged['team_x'].dropna().unique()],
                    placeholder="Select a team",
                    clearable=True,
                    value=default_team,  # Set the default to 'Argentina'
                    style={'width': '100%'}  # Adjust width of dropdown
                )
            ], style={'margin': '10px'}), width=4),

            dbc.Col(html.Div([
                html.Label('Select players:', style={'fontWeight': 'bold', 'color': 'black'}),
                dcc.Dropdown(
                    id='player-dropdown',
                    placeholder="Select players",
                    multi=True,
                    clearable=True,
                    value=[default_highlighted_player],  # Pre-select the best player
                    style={'width': '100%'}  # Adjust width of dropdown
                )
            ], style={'margin': '10px'}), width=4),

            dbc.Col(html.Div([
                html.Label('Select player to highlight:', style={'fontWeight': 'bold', 'color': 'black'}),
                dcc.Dropdown(
                    id='highlight-player-dropdown',
                    placeholder="Select a player to highlight",
                    clearable=True,
                    value=default_highlighted_player,  # Highlight the best player by default
                    style={'width': '100%'}  # Adjust width of dropdown
                )
            ], style={'margin': '10px'}), width=4),
        ]),
        html.Div([
            dcc.Graph(id='player-card', style={"height": "70vh", "width": "40%", "backgroundColor": "#9bc2cf"}),
            dcc.Graph(id='radar-chart', style={"height": "70vh", "width": "60%", "backgroundColor": "white"})  # Set white background for radar chart
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'})
    ]
)

@app.callback(
    Output('player-dropdown', 'options'),
    Output('player-dropdown', 'value'),
    Output('highlight-player-dropdown', 'value'),
    Input('team-dropdown', 'value')
)
def set_player_options(selected_team):
    if not selected_team or selected_team == 'No Team':
        player_options = [{'label': player, 'value': player} for player in df_merged['player'].unique()]
        return player_options, [default_highlighted_player], default_highlighted_player
    filtered_df = df_merged[df_merged['team_x'] == selected_team]
    player_options = [{'label': player, 'value': player} for player in filtered_df['player'].unique()]
    best_player_team = filtered_df.loc[filtered_df['total_performance'].idxmax()]['player']
    return player_options, [option['value'] for option in player_options], best_player_team

@app.callback(
    Output('highlight-player-dropdown', 'options'),
    Input('player-dropdown', 'value')
)
def set_highlight_player_options(selected_players):
    if not selected_players:
        return []
    return [{'label': player, 'value': player} for player in selected_players]

# @app.callback(
#     Output('player-card', 'figure'),
#     Input('team-dropdown', 'value'),
#     Input('player-dropdown', 'value'),
#     Input('highlight-player-dropdown', 'value')
# )
# def update_player_card(selected_team, selected_players, highlighted_player):
#     if highlighted_player:
#         # If a player is highlighted, show their information
#         best_player = df_merged[df_merged['player'] == highlighted_player].iloc[0]
#         title = f'<b>Selected Player Info: <b>{best_player["player"]}'
#     elif not selected_team or selected_team == 'No Team':
#         if selected_players:
#             # Determine the best player based on the total performance from the selected players
#             filtered_df = df_merged[df_merged['player'].isin(selected_players)]
#             best_player = filtered_df.loc[filtered_df['total_performance'].idxmax()]
#             title = f'<b>Best Selected Players Info: <b>{best_player["player"]}'
#         else:
#             # Determine the best player based on the total performance from the entire dataset
#             best_player = df_merged.loc[df_merged['total_performance'].idxmax()]
#             title = f'<b>Tournament Best Player Info: <b>{best_player["player"]}'
#     else:
#         # Filter the dataframe based on the selected team
#         filtered_df = df_merged[df_merged['team_x'] == selected_team]
# 
#         if filtered_df.empty:
#             return go.Figure()
# 
#         # Determine the best player based on the total performance for the selected team
#         best_player = filtered_df.loc[filtered_df['total_performance'].idxmax()]
#         title = f'<b>Best Player Info: <b>{best_player["player"]}'
# 
#     # Convert position codes to full names
#     position_map = {
#         'FW': 'Forward',
#         'DF': 'Defence',
#         'GK': 'Goalkeeper',
#         'MF': 'Midfielder'
#     }
#     position_full = position_map.get(best_player['position_x'], best_player['position_x'])
# 
#     # Create the figure
#     fig = go.Figure()
# 
#     annotations = []
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=0.1, y=0.9,
#                             text='<b>Name       : </b>' + str(best_player['player']),
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=0.1, y=0.7,
#                             text='<b>Position   : </b>' + position_full,
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=0.1, y=0.3,
#                             text='<b>Team       : </b>' + str(best_player['team_x']),
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=0.1, y=0.1,
#                             text='<b>Year of Birth : </b>' + str(best_player['birth_year_x']),
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=1, y=0.9,
#                             text='<b>Passing   : </b>' + str(round(best_player['passing_commulative_performance'], 2)),
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=1, y=0.7,
#                             text='<b>Shooting   : </b>' + str(round(best_player['shooting_commulative_performance'], 2)),
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=1, y=0.3,
#                             text='<b>Defence   : </b>' + str(round(best_player['defence_commulative_performance'], 2)),
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
#     annotations.append(dict(xref='paper', yref='paper',
#                             x=1, y=0.1,
#                             text='<b>Possession: </b>' + str(round(best_player['possession_commulative_performance'], 2)),
#                             font=dict(family='Arial', size=16, color='#161616'),
#                             showarrow=False))
# 
#     fig.update_layout(title=title,
#                       titlefont={'size': 24},
#                       font_family='San Serif',
#                       width=650, height=300,
#                       template="simple_white",
#                       showlegend=False,
#                       paper_bgcolor="#9bc2cf",
#                       plot_bgcolor="white",
#                       font=dict(color='black'),
#                       legend=dict(orientation="v",
#                                   y=1,
#                                   yanchor="bottom",
#                                   x=1.0,
#                                   xanchor="right",)
#                       )
#     fig.update_layout(annotations=annotations)
#     fig.update_xaxes(visible=False)
#     fig.update_yaxes(visible=False)
# 
#     return fig


@app.callback(
    Output('player-card', 'figure'),
    Input('team-dropdown', 'value'),
    Input('player-dropdown', 'value'),
    Input('highlight-player-dropdown', 'value')
)
def update_player_card(selected_team, selected_players, highlighted_player):
    if highlighted_player:
        # If a player is highlighted, show their information
        best_player = df_merged[df_merged['player'] == highlighted_player].iloc[0]
        title = f'<b>Selected Player Info: <b>{best_player["player"]}'
    elif not selected_team or selected_team == 'No Team':
        if selected_players:
            # Determine the best player based on the total performance from the selected players
            filtered_df = df_merged[df_merged['player'].isin(selected_players)]
            best_player = filtered_df.loc[filtered_df['total_performance'].idxmax()]
            title = f'<b>Best Selected Players Info: <b>{best_player["player"]}'
        else:
            # Determine the best player based on the total performance from the entire dataset
            best_player = df_merged.loc[df_merged['total_performance'].idxmax()]
            title = f'<b>Tournament Best Player Info: <b>{best_player["player"]}'
    else:
        # Filter the dataframe based on the selected team
        filtered_df = df_merged[df_merged['team_x'] == selected_team]

        if filtered_df.empty:
            return go.Figure()

        # Determine the best player based on the total performance for the selected team
        best_player = filtered_df.loc[filtered_df['total_performance'].idxmax()]
        title = f'<b>Best Player Info: <b>{best_player["player"]}'

    # Convert position codes to full names
    position_map = {
        'FW': 'Forward',
        'DF': 'Defence',
        'GK': 'Goalkeeper',
        'MF': 'Midfielder'
    }
    position_full = position_map.get(best_player['position_x'], best_player['position_x'])

    # Create the figure
    fig = go.Figure()

    annotations = []
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.9,
                            text='<b>Name       : </b>' + str(best_player['player']),
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.7,
                            text='<b>Position   : </b>' + position_full,
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.3,
                            text='<b>Team       : </b>' + str(best_player['team_x']),
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.1,
                            text='<b>Year of Birth : </b>' + str(best_player['birth_year_x']),
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=1, y=0.9,
                            text='<b>Passing   : </b>' + str(round(best_player['passing_commulative_performance'], 2)),
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=1, y=0.7,
                            text='<b>Shooting   : </b>' + str(round(best_player['shooting_commulative_performance'], 2)),
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=1, y=0.3,
                            text='<b>Defence   : </b>' + str(round(best_player['defence_commulative_performance'], 2)),
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=1, y=0.1,
                            text='<b>Possession: </b>' + str(round(best_player['possession_commulative_performance'], 2)),
                            font=dict(family='Arial', size=16, color='#161616'),
                            showarrow=False))

    fig.update_layout(title=title,
                      titlefont={'size': 24},
                      font_family='San Serif',
                      width=650, height=300,
                      template="simple_white",
                      showlegend=False,
                      paper_bgcolor="#9bc2cf",
                      plot_bgcolor="white",
                      font=dict(color='black'),
                      images=[dict(
                          source="CleanedData/other.png",  # URL to the default player image
                          xref="paper", yref="paper",
                          x=0.5, y=0.9,
                          sizex=0.4, sizey=0.4,
                          xanchor="center", yanchor="middle"
                      )],
                      legend=dict(orientation="v",
                                  y=1,
                                  yanchor="bottom",
                                  x=1.0,
                                  xanchor="right",)
                      )
    fig.update_layout(annotations=annotations)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)

    return fig




@app.callback(
    Output('radar-chart', 'figure'),
    [Input('team-dropdown', 'value'),
     Input('player-dropdown', 'value'),
     Input('highlight-player-dropdown', 'value')]
)
def update_radar_chart(selected_team, selected_players, highlighted_player):
    if not selected_players:
        return go.Figure()

    if selected_team == 'No Team':
        filtered_df = df_merged
    else:
        filtered_df = df_merged[df_merged['team_x'] == selected_team]

    categories = [
        'defence_commulative_performance',
        'passing_commulative_performance',
        'shooting_commulative_performance',
        'possession_commulative_performance',
        'defence_commulative_performance'  # Repeat the first category to close the radar chart loop
    ]
    categories_labels = ['Defence', 'Passing', 'Shooting', 'Possession', 'Defence']

    radial_ticks = np.arange(0, 100, 4)  # Adjust tick values

    fig = go.Figure()

    # Add the average performance line
    fig.add_trace(go.Scatterpolar(
        r=[average_performance[cat] for cat in categories],
        theta=categories_labels,
        name='Tournament Average',
        mode='lines',
        line_color='green',
        line_shape='spline',
        line_smoothing=0.8,
        opacity=0.8,
        line_width=2,
        hovertemplate='<b>%{theta}:%{r}<br></b><br>'
    ))

    for player in selected_players:
        player_data = filtered_df[filtered_df['player'] == player]
        if not player_data.empty:
            fig.add_trace(go.Scatterpolar(
                r=[player_data[cat].values[0] for cat in categories],
                theta=categories_labels,
                name=player,
                mode='lines',
                line_color='orange' if player == highlighted_player else 'black',
                line_shape='spline',
                line_smoothing=0.8,
                opacity=1 if player == highlighted_player else 0.6,
                line_width=3 if player == highlighted_player else 1,
                hovertemplate='<b>%{theta}:%{r}<br></b><br>'
            ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, tickvals=radial_ticks),
            bgcolor='white',  # Set the background color of the radar chart to white
            radialaxis_tickfont_color='darkgrey',
            angularaxis_color='black',
            angularaxis_showline=False,
            radialaxis_showline=False,
            radialaxis_gridcolor='#F2F2F2'
        ),
        title=f'<b>Performance Metrics<b>',
        font=dict(size=15, color='black'),
        paper_bgcolor="#9bc2cf",  # Ensure the radar chart's background is light blue
        plot_bgcolor="white"  # Ensure the radar chart's plot background is white
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8046)


In [114]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, exceptions
import dash_bootstrap_components as dbc  # Import Dash Bootstrap Components
import numpy as np
from PIL import Image
import os

# Load your data
df_radar = pd.read_csv("CleanedData/player_radar.csv")
df_stats = pd.read_csv("CleanedData/player_stats_cleaned.csv")

# Merge the datasets on the 'player' column without adding suffixes
merged_df = pd.merge(df_stats, df_radar, on='player', how='outer', suffixes=('', '_drop'))

# Drop columns with duplicate names from the second dataframe
df = merged_df.drop(merged_df.filter(regex='_drop$').columns, axis=1)

df_merged = df.copy()

# Calculate performance metrics
df_merged['passing_commulative_performance'] = (
    0.05 * df_merged['passes_completed'] +
    1.5 * df_merged['assists'] +
    1.5 * df_merged['assisted_shots'] +
    0.2 * df_merged['passes_into_final_third'] +
    0.4 * df_merged['passes_into_penalty_area'] +
    0.3 * df_merged['crosses_into_penalty_area'] +
    0.1 * df_merged['progressive_passes']
)

df_merged['shooting_commulative_performance'] = (
    3 * df_merged['goals'] +
    0.7 * df_merged['shots_on_target'] +
    0.6 * df_merged['shots'] +
    0.4 * df_merged['shots_free_kicks'] +
    df_merged['pens_made']
)

df_merged['defence_commulative_performance'] = (
    -0.2 * df_merged['dribbled_past'] -
    df_merged['errors'] +
    0.5 * df_merged['blocks'] +
    0.75 * df_merged['blocked_shots'] +
    0.75 * df_merged['blocked_passes'] +
    df_merged['tackles_interceptions'] +
    0.2 * df_merged['clearances']
)

df_merged['possession_commulative_performance'] = (
    0.05 * df_merged['touches'] +
    0.2 * df_merged['touches_att_3rd'] +
    0.5 * df_merged['touches_att_pen_area'] +
    1 * df_merged['dribbles_completed'] +
    0.1 * df_merged['progressive_passes_received'] +
    -0.2 * df_merged['miscontrols'] +
    -0.5 * df_merged['dispossessed']
)

df_merged['total_performance'] = (
    df_merged['passing_commulative_performance'] +
    df_merged['shooting_commulative_performance'] +
    df_merged['defence_commulative_performance'] +
    df_merged['possession_commulative_performance']
)

# Determine the best player based on total performance
best_player_overall = df_merged.loc[df_merged['total_performance'].idxmax()]
default_team = 'Argentina'  # Set to 'Argentina' to show Argentina players initially
default_highlighted_player = best_player_overall['player']

# Calculate the average performance metrics for all players
average_performance = df_merged[['defence_commulative_performance', 'passing_commulative_performance', 'shooting_commulative_performance', 'possession_commulative_performance']].mean()

# Resize the image
image_path = "CleanedData/other.png"
image = Image.open(image_path)
resized_image = image.resize((300, 300))  # Resize to 300x300 for better fit

# Save the resized image to the assets folder
assets_folder = "assets"
os.makedirs(assets_folder, exist_ok=True)
resized_image_path = os.path.join(assets_folder, "resized_other.png")
resized_image.save(resized_image_path)

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    style={'backgroundColor': '#9bc2cf', 'minHeight': '100vh'},  # Set light blue background for the entire page
    children=[
        dbc.Row([
            dbc.Col(html.Div([
                html.Label('Select team:', style={'fontWeight': 'bold', 'color': 'black'}),
                dcc.Dropdown(
                    id='team-dropdown',
                    options=[{'label': team, 'value': team} for team in df_merged['team'].dropna().unique()],
                    placeholder="Select a team",
                    clearable=True,
                    value=default_team,  # Set the default to 'Argentina'
                    style={'width': '100%'}  # Adjust width of dropdown
                )
            ], style={'margin': '10px'}), width=4),

            dbc.Col(html.Div([
                html.Label('Select players:', style={'fontWeight': 'bold', 'color': 'black'}),
                dcc.Dropdown(
                    id='player-dropdown',
                    placeholder="Select players",
                    multi=True,
                    clearable=True,
                    value=[default_highlighted_player],  # Pre-select the best player
                    style={'width': '100%'}  # Adjust width of dropdown
                )
            ], style={'margin': '10px'}), width=4),

            dbc.Col(html.Div([
                html.Label('Select player to highlight:', style={'fontWeight': 'bold', 'color': 'black'}),
                dcc.Dropdown(
                    id='highlight-player-dropdown',
                    placeholder="Select a player to highlight",
                    clearable=True,
                    value=default_highlighted_player,  # Highlight the best player by default
                    style={'width': '100%'}  # Adjust width of dropdown
                )
            ], style={'margin': '10px'}), width=4),
        ]),
        html.Div([
            dcc.Graph(id='player-card', style={"height": "70vh", "width": "40%", "backgroundColor": "#9bc2cf"}),
            dcc.Graph(id='radar-chart', style={"height": "70vh", "width": "60%", "backgroundColor": "white"})  # Set white background for radar chart
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'})
    ]
)

@app.callback(
    Output('player-dropdown', 'options'),
    Output('player-dropdown', 'value'),
    Output('highlight-player-dropdown', 'value'),
    Input('team-dropdown', 'value')
)
def set_player_options(selected_team):
    if not selected_team:
        player_options = [{'label': player, 'value': player} for player in df_merged['player'].unique()]
        return player_options, [default_highlighted_player], default_highlighted_player
    filtered_df = df_merged[df_merged['team'] == selected_team]
    player_options = [{'label': player, 'value': player} for player in filtered_df['player'].unique()]
    best_player_team = filtered_df.loc[filtered_df['total_performance'].idxmax()]['player']
    return player_options, [option['value'] for option in player_options], best_player_team

@app.callback(
    Output('highlight-player-dropdown', 'options'),
    Input('player-dropdown', 'value')
)
def set_highlight_player_options(selected_players):
    if not selected_players:
        return []
    return [{'label': player, 'value': player} for player in selected_players]

@app.callback(
    Output('player-card', 'figure'),
    Input('team-dropdown', 'value'),
    Input('player-dropdown', 'value'),
    Input('highlight-player-dropdown', 'value')
)
def update_player_card(selected_team, selected_players, highlighted_player):
    if highlighted_player:
        # If a player is highlighted, show their information
        best_player = df_merged[df_merged['player'] == highlighted_player].iloc[0]
        title = f'<b>Selected Player Info: <b>{best_player["player"]}'
    elif not selected_team or selected_team == 'No Team':
        if selected_players:
            # Determine the best player based on the total performance from the selected players
            filtered_df = df_merged[df_merged['player'].isin(selected_players)]
            best_player = filtered_df.loc[filtered_df['total_performance'].idxmax()]
            title = f'<b>Best Selected Players Info: <b>{best_player["player"]}'
        else:
            # Determine the best player based on the total performance from the entire dataset
            best_player = df_merged.loc[df_merged['total_performance'].idxmax()]
            title = f'<b>Tournament Best Player Info: <b>{best_player["player"]}'
    else:
        # Filter the dataframe based on the selected team
        filtered_df = df_merged[df_merged['team'] == selected_team]

        if filtered_df.empty:
            return go.Figure()

        # Determine the best player based on the total performance for the selected team
        best_player = filtered_df.loc[filtered_df['total_performance'].idxmax()]
        title = f'<b>Best Player Info: <b>{best_player["player"]}'

    # Convert position codes to full names
    position_map = {
        'FW': 'Forward',
        'DF': 'Defence',
        'GK': 'Goalkeeper',
        'MF': 'Midfielder'
    }
    position_full = position_map.get(best_player['position'], best_player['position'])

    # Create the figure
    fig = go.Figure()
    
    # Define the spacing between each line
    spacing = 0.1
    
    # Create annotations with equal spacing
    annotations = []
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.6,
                            text='<b>Name       : </b>' + str(best_player['player']),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.6 - spacing,
                            text='<b>Position   : </b>' + position_full,
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.6 - 2 * spacing,
                            text='<b>Team       : </b>' + str(best_player['team']),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.6 - 3.2 * spacing,
                            text='<b>Club       : </b>' + str(best_player['club']),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.1, y=0.6 - 4.1 * spacing,
                            text='<b>Year of Birth : </b>' + str(best_player['birth_year']),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.7, y=0.72,
                            text='<b>Goals       : </b>' + str(best_player['goals']),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.7, y=0.7 - spacing,
                            text='<b>Assists     : </b>' + str(best_player['assists']),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.7, y=0.7 - 2 * spacing,
                            text='<b>Passing   : </b>' + str(round(best_player['passing_commulative_performance'], 2)),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.7, y=0.7 - 3 * spacing,
                            text='<b>Shooting  : </b>' + str(round(best_player['shooting_commulative_performance'], 2)),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.7, y=0.7 - 4.2 * spacing,
                            text='<b>Defense   : </b>' + str(round(best_player['defence_commulative_performance'], 2)),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    annotations.append(dict(xref='paper', yref='paper',
                            x=0.7, y=0.7 - 5.1 * spacing,
                            text='<b>Possession: </b>' + str(round(best_player['possession_commulative_performance'], 2)),
                            font=dict(family='Arial', size=14, color='#161616'),
                            showarrow=False))
    
    fig.update_layout(
        title=title,
        titlefont={'size': 24},
        font_family='San Serif',
        width=650, height=500,
        template="simple_white",
        showlegend=False,
        paper_bgcolor="#9bc2cf",
        plot_bgcolor="white",
        font=dict(color='black'),
        images=[dict(
            source=app.get_asset_url('resized_other.png'),  # Use the resized image from assets folder
            xref="paper", yref="paper",
            x=0.11, y=1,
            sizex=0.25, sizey=0.35,
            xanchor="center", yanchor="top"
        )],
        legend=dict(orientation="v",
                    y=1,
                    yanchor="bottom",
                    x=1.0,
                    xanchor="right",)
    )
    fig.update_layout(annotations=annotations)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)


    return fig



@app.callback(
    Output('radar-chart', 'figure'),
    [Input('team-dropdown', 'value'),
     Input('player-dropdown', 'value'),
     Input('highlight-player-dropdown', 'value')]
)
def update_radar_chart(selected_team, selected_players, highlighted_player):
    if not selected_players:
        return go.Figure()

    if selected_team == 'No Team' or not selected_team:
        filtered_df = df_merged
    else:
        filtered_df = df_merged[df_merged['team'] == selected_team]

    categories = [
        'defence_commulative_performance',
        'passing_commulative_performance',
        'shooting_commulative_performance',
        'possession_commulative_performance',
        'defence_commulative_performance'  # Repeat the first category to close the radar chart loop
    ]
    categories_labels = ['Defence', 'Passing', 'Shooting', 'Possession', 'Defence']

    radial_ticks = np.arange(0, 100, 4)  # Adjust tick values

    fig = go.Figure()

    # Add the average performance line
    fig.add_trace(go.Scatterpolar(
        r=[average_performance[cat] for cat in categories],
        theta=categories_labels,
        name='Tournament Average',
        mode='lines',
        line_color='green',
        line_shape='spline',
        line_smoothing=0.8,
        opacity=0.8,
        line_width=2,
        hovertemplate='<b>%{theta}:%{r}<br></b><br>'
    ))

    for player in selected_players:
        player_data = filtered_df[filtered_df['player'] == player]
        if not player_data.empty:
            fig.add_trace(go.Scatterpolar(
                r=[player_data[cat].values[0] for cat in categories],
                theta=categories_labels,
                name=player,
                mode='lines',
                line_color='orange' if player == highlighted_player else 'black',
                line_shape='spline',
                line_smoothing=0.8,
                opacity=1 if player == highlighted_player else 0.6,
                line_width=3 if player == highlighted_player else 1,
                hovertemplate='<b>%{theta}:%{r}<br></b><br>'
            ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, tickvals=radial_ticks),
            bgcolor='white',  # Set the background color of the radar chart to white
            radialaxis_tickfont_color='darkgrey',
            angularaxis_color='black',
            angularaxis_showline=False,
            radialaxis_showline=False,
            radialaxis_gridcolor='#F2F2F2'
        ),
        title=f'<b>Performance Metrics<b>',
        font=dict(size=15, color='black'),
        paper_bgcolor="#9bc2cf",  # Ensure the radar chart's background is light blue
        plot_bgcolor="white"  # Ensure the radar chart's plot background is white
    )

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, port=8046)


In [117]:
import threading
from IPython.display import display, HTML

# Function to run the Dash app
def run_dash():
    app.run_server(debug=False, use_reloader=False, port=8046)

# Start the Dash app in a separate thread
threading.Thread(target=run_dash).start()

# Display the link to open the app in the browser
display(HTML(f"""
    <a href="http://127.0.0.1:8046/" target="_blank">
        Open Dash App
    </a>
"""))